In [6]:
# You are part of a geospatial AI team working with satellite imagery to monitor land use changes. Your task is to build a model that classifies satellite images into three categories:
# Urban
# Forest
# Water
# Due to limited labeled data and compute resources, training a deep CNN from scratch is not practical.

# Challenge
# Apply transfer learning using a pre-trained ResNet50 model (trained on ImageNet) to classify satellite images into the three land cover types.

import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import Subset, DataLoader
import os

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
#Downloading and fltering
# 1. Download the full 10-class dataset
full_dataset = datasets.EuroSAT(root='data/', download=True, transform=transform)

# 2. Identify the labels for our 3 target classes
# EuroSAT labels: 'Forest': 1, 'Residential': 7, 'River': 8 (Commonly)
# Let's find them dynamically to be safe:
targets = {v: k for k, v in full_dataset.class_to_idx.items()}
print("Full Dataset Classes:", targets)

# We want: Forest, Residential (Urban), and River (Water)
wanted_classes = ['Forest', 'Residential', 'River']
wanted_indices = [full_dataset.class_to_idx[cls] for cls in wanted_classes]
print(wanted_indices)

# 3. Create a Filter (Subset)
# We find the position of every image that belongs to our 3 classes
indices = [i for i, label in enumerate(full_dataset.targets) if label in wanted_indices]

# Now we have a smaller dataset with ONLY our 3 classes
my_subset = Subset(full_dataset, indices)

# Create the Loader (The conveyor belt that feeds the model)
train_loader = DataLoader(my_subset, batch_size=8, shuffle=True)

Full Dataset Classes: {0: 'AnnualCrop', 1: 'Forest', 2: 'HerbaceousVegetation', 3: 'Highway', 4: 'Industrial', 5: 'Pasture', 6: 'PermanentCrop', 7: 'Residential', 8: 'River', 9: 'SeaLake'}


In [9]:
#Model surgery
# Load pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Freeze the "Eyes" (Don't train the backbone)
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer (fc)
# We change out_features from 1000 to 3
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

# Transfer model to GPU if you have one
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# Training Loop
for epoch in range(2): # Just 2 loops for this example
    model.train()
    for images, labels in train_loader:
        # Relabel the targets to 0, 1, 2 for our 3-class model
        # (Since original labels were 1, 7, 8)
        new_labels = torch.tensor([wanted_indices.index(l) for l in labels]).to(device)

        images = images.to(device)

        # 1. Forward pass (The Guess)
        outputs = model(images)
        loss = criterion(outputs, new_labels)

        # 2. Backward pass (The Fix)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} complete. Loss: {loss.item():.4f}")

KeyboardInterrupt: 